In [1]:
import time
import json
from collections import namedtuple
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

In [37]:
Credentials = namedtuple('Credentials', ['username', 'password'])


def init_driver(driver_path):
    driver = webdriver.Chrome(driver_path)
    return driver

def get_credentials(path):
    with open(path, "r") as f:
        lines = f.readlines()
        account = Credentials(username=lines[0], password=lines[1])
    return account

def login(driver, credentials):
    driver.get("http://www.glassdoor.com/profile/login_input.htm")
    try:
        user_field = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "username")))
        pw_field = driver.find_element_by_class_name("signin-password")
        login_button = driver.find_element_by_id("signInBtn")
        user_field.send_keys(credentials.username)
        user_field.send_keys(Keys.TAB)
        time.sleep(1)
        pw_field.send_keys(credentials.password)
        time.sleep(1)
        login_button.click()
        time.sleep(3)
    except TimeoutException:
        print("TimeoutException! Username/password field or login button not found on glassdoor.com")

def get_total_pages(driver, url):
    driver.get(url)
    total_rev_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.eiCell.cell.reviews.active'))
    total_rev_element = WebDriverWait(driver, 10).until(total_rev_present)
    total_rev_str = total_rev_element.find_element_by_css_selector('.num.h2').text
    total_rev = int(total_rev_str)
    total_pages = total_rev // reviews_per_page
    return total_pages

def get_datetime(review):
    datetime = review.find("time").attrs['datetime']
    return datetime

def get_ratings(review):
    themes = ['Work Life Balance',
              'Culture and Values',
              'Career Opportunities',
              'Compensation and Benefits',
              'Senior Management']
    ratings_html = review.find_all('span', {'class': "gdBars gdRatings med "})
    ratings = {theme: int(float(rating.attrs['title'])) for theme, rating in zip(themes, ratings_html)}
    return ratings

def get_title(review):
    title = review.find('span', {'class': 'authorJobTitle middle reviewer'}).getText()
    return title

def get_location(review):
    location = review.find('span', {'class': 'authorLocation middle'}).getText()
    return location

def get_recommendations(review):
    options = {'CE'}
    t = review.find('div', {'class': 'flex-grid recommends'})
    recommendations_html = t.find_all('span', class_='middle')
    recommendations = {'Recommendation {}'.format(idx+1): rec.getText()
                        for idx, rec in enumerate(recommendations_html)}
    return recommendations

In [3]:
chromium_path = "/usr/local/bin/chromedriver"
reviews_per_page = 10
companyName = "sandvik"
companyURL = "https://www.glassdoor.com/Reviews/Sandvik-Reviews-E10375.htm"

In [4]:
driver = init_driver(chromium_path)
credentials = get_credentials("account.txt")
login(driver=driver, credentials=credentials)

In [ ]:
total_pages = get_total_pages(driver=driver, url=companyURL)

In [ ]:
HTML = driver.page_source
soup = BeautifulSoup(HTML, "html.parser")
reviews = soup.find_all("li", { "class" : ["empReview", "padVert"] })

In [ ]:
driver.get(companyURL)

In [38]:
total_pages = get_total_pages(driver=driver, url=companyURL)

In [ ]:
d = []

for page_no in range(1, total_pages):
    currentURL = '{}_P{}.htm'.format(companyURL[:companyURL.rfind('.')], page_no)
    driver.get(currentURL)
    driver.execute_script("window.scrollTo(0, 100)")
    time.sleep(randint(5,10))
    HTML = driver.page_source
    soup = BeautifulSoup(HTML, "html.parser")
    reviews = soup.find_all("li", { "class" : ["empReview", "padVert"] })
    dates = parse_reviews_HTML(reviews)
    d.extend(dates)

In [ ]:
review = reviews[1]

In [ ]:
review.find("time").attrs['datetime']

In [ ]:
review

In [ ]:
t = review.find('div', {'class': 'flex-grid recommends'})

In [ ]:
t.find_all('span', class_='middle')[0].getText()

In [ ]:
t = review.find('div', {'class': 'flex-grid recommends'})
recommendations_html = t.find_all('span', class_='middle')
recommendations = {'Recommendation {}'.format(idx+1): rec.getText()
                   for idx, rec in enumerate(recommendations_html)}

In [ ]:
recommendations

In [ ]:
get_title(review)

In [13]:
driver.get(companyURL)

In [36]:
WebDriverWait(driver, 1).until()

<selenium.webdriver.remote.webelement.WebElement (session="8252f701b511969411a0a6fdecd75162", element="0.1905742344207566-1")>

In [16]:
EC.presence_of_element_located((By.CLASS_NAME, '.eiCell.cell.reviews.active'))

In [29]:
element = driver.find_element_by_css_selector('.eiCell.cell.reviews.active')

In [34]:
e = element.find_element_by_css_selector('.num.h2')

In [35]:
e.text

'294'

In [18]:
driver.findElement(By.cssSelector("eiCell cell reviews active"))

AttributeError: 'WebDriver' object has no attribute 'findElement'

In [17]:
driver.find_element_by_class_name('eiCell cell reviews active')

InvalidSelectorException: Message: invalid selector: Compound class names not permitted
  (Session info: chrome=62.0.3202.94)
  (Driver info: chromedriver=2.34.522913 (36222509aa6e819815938cbf2709b4849735537c),platform=Linux 4.4.0-104-generic x86_64)
